In [4]:
#importing required libraries
import os
import numpy as np
import pandas as pd
from optparse import OptionParser
import shutil
from subprocess import call
from optparse import OptionParser

In [5]:
#required functions
#fixing '5E54' becoming '5.00E+54' issue
def fix_PDB_ID(df):
    for i, j in enumerate(df['PDB_ID']):
        print (j)
        if len(str(j))>4:
            if '-' in j:
                X= ''.join(j.split('-')).upper()
                df.loc[i, 'PDB_ID']= X
            else:
                if '.' in j:
                    x= str(j)
                    x1= x.split('.')[0]
                    x2= x.split('+')[1]
                    X= x1+'E'+x2
                    print (X)
                    df.loc[i, 'PDB_ID']= X
                else:
                    x= str(j)
                    x1= x.split('+')[0]
                    x2= x.split('+')[1]
                    X= x1+ x2
                    print (X)
                    df.loc[i, 'PDB_ID']= X
                    
    return (df)

# combining all results (csvs) into one

In [6]:
# Specify the root directory where you want to start the search
root_dir = '../data/Input'  # Replace with your directory

# List to hold DataFrames
dfs = []

# Walk through all directories and subdirectories
for dirpath, dirnames, filenames in os.walk(root_dir):
    for file in filenames:
        if file.endswith('.csv'):  # Check if the file is a CSV
            print (file)
            # Get the full file path
            file_path = os.path.join(dirpath, file)
            # Read the CSV file into a DataFrame and append it to the list
            d = pd.read_csv(file_path)
            #checking if there are two rows of columns
            unnamed_columns = [col for col in d.columns if col is None or "Unnamed" in str(col)]
            if len(unnamed_columns)>0:
                d.columns = d.iloc[0]  # Set the second row (index 1) as the header
                d = d[1:].reset_index(drop=True)
            else:
                pass
            #updating the 'Refinement Resolution' to 'Resolution'
            d.columns = d.columns.str.replace("Refinement ", "", regex=False)

            #replacing ' ' with '_' from all columns name
            d.columns = d.columns.str.replace(' ', '_') 
            d1 = d[['PDB_ID', 'Experimental_Method', 'Resolution_(Å)']].copy()
            print (d1.shape)
            print (d1.columns)
            dfs.append(d1)
            

# Concatenate all DataFrames into one
D = pd.concat(dfs, ignore_index=True)

D= fix_PDB_ID(D)
print (D.shape)

rcsb_pdb_custom_report_ba0be092afc119988ec6c834c6ec500b_7501-8748.csv
(25586, 3)
Index(['PDB_ID', 'Experimental_Method', 'Resolution_(Å)'], dtype='object', name=0)
rcsb_pdb_custom_report_ba0be092afc119988ec6c834c6ec500b_5001-7500.csv
(57731, 3)
Index(['PDB_ID', 'Experimental_Method', 'Resolution_(Å)'], dtype='object', name=0)
rcsb_pdb_custom_report_ba0be092afc119988ec6c834c6ec500b_0001-2500.csv
(13947, 3)
Index(['PDB_ID', 'Experimental_Method', 'Resolution_(Å)'], dtype='object', name=0)
rcsb_pdb_custom_report_ba0be092afc119988ec6c834c6ec500b_2501-5000.csv
(53177, 3)
Index(['PDB_ID', 'Experimental_Method', 'Resolution_(Å)'], dtype='object', name=0)
8PSO
nan
nan
nan
nan
8PSQ
nan
nan
nan
8PSS
nan
nan
nan
8PSU
nan
nan
nan
8PSX
nan
nan
nan
nan
8PSZ
nan
nan
nan
nan
8PT2
nan
nan
nan
8PT6
nan
nan
nan
nan
8PT7
nan
nan
nan
8PTG
nan
8PTH
nan
nan
nan
8PTJ
nan
nan
nan
8PTX
nan
nan
nan
8PTZ
nan
nan
nan
8PU0
nan
nan
nan
8PV1
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan


In [7]:
#because each rows in D are entities in the structures
#repeation of same structure (PDB_ID) is possible.
#now we are removing the duplicates, therefore, we will have detail information for each PDB_ID one time
D1= D.drop_duplicates(subset=['PDB_ID'])
D1.index = np.arange(0, len(D1))

#filtering out empty cell at 'PDB_ID' columns
D2 = D1.dropna(subset=['PDB_ID'])

#sorting the dataframe by the PDB_ID
D3 = D2.sort_values(by='PDB_ID')

In [8]:
print (D3.shape)

(8748, 3)


In [9]:
D3

,PDB_ID,Experimental_Method,Resolution_(Å)
3749,124D,SOLUTION NMR,NaN
3750,157D,X-RAY DIFFRACTION,1.8
3751,165D,X-RAY DIFFRACTION,1.55
3752,176D,SOLUTION NMR,NaN
3753,17RA,SOLUTION NMR,NaN
...,...,...,...
1244,9QEH,ELECTRON MICROSCOPY,2.08
1245,9QIW,ELECTRON MICROSCOPY,3.04
1246,9QRP,X-RAY DIFFRACTION,2.7
1247,9QT5,ELECTRON MICROSCOPY,3.13


In [15]:
D3_filtered=D3[D3['Experimental_Method']== 'SOLUTION NMR']
D3_filtered.index= np.arange(0, len(D3_filtered))

In [16]:
D3_filtered

,PDB_ID,Experimental_Method,Resolution_(Å)
0,124D,SOLUTION NMR,NaN
1,176D,SOLUTION NMR,NaN
2,17RA,SOLUTION NMR,NaN
3,1A1T,SOLUTION NMR,NaN
4,1A3M,SOLUTION NMR,NaN
...,...,...,...
769,9IOS,SOLUTION NMR,NaN
770,9IOU,SOLUTION NMR,NaN
771,9JVN,SOLUTION NMR,NaN
772,9KAD,SOLUTION NMR,NaN


In [23]:
R= 3.0

In [24]:
#filter out all base pair within a desired resolution cut-off
#apply resolution cut-off (3.2 A)
#removing any empty cells
D4= D3[~D3['Resolution_(Å)'].isna()]
D4.index = np.arange(0, len(D4))

#converting all entries in 'Resolution_(Å)' to float
D5= D4.copy()
D5['Resolution_(Å)'] = pd.to_numeric(D5['Resolution_(Å)'], errors='coerce')

#applying resolution cut-off
D6= D5[(D5['Resolution_(Å)']< float(R)) | (D5['Resolution_(Å)']== float(R)) ]
D6.index = np.arange(0, len(D6))

In [25]:
D6.shape

(4121, 3)

In [8]:
D6.columns

Index(['PDB_ID', 'Experimental_Method', 'Resolution_(Å)'], dtype='object', name=0)

In [26]:
D6.to_csv('structures_jun_2025_3A_res_cutoff.csv', index= False)

In [10]:
os.chdir('/Users/sharear/Documents/sky/RESEARCH_&_GRAD_SCHOOL/Penn_State/cifs_jsons/RCSB_search_outputs/apr_2025')

In [14]:
str_files=[]
str_dir= '/Users/sharear/Documents/sky/RESEARCH_&_GRAD_SCHOOL/Penn_State/cifs_jsons/str_files'
for i, j in enumerate(D6['PDB_ID']):
    str_files.append(j)
print (str_files)

['157D', '165D', '1A34', '1A9N', '1AQ3', '1AQ4', '1ASY', '1ASZ', '1AV6', '1B23', '1B2M', '1B7F', '1BMV', '1BR3', '1C0A', '1C9S', '1CGM', '1CSL', '1CVJ', '1CWP', '1D4R', '1DDL', '1DDY', '1DFU', '1DI2', '1DK1', '1DQF', '1DQH', '1DRZ', '1DUH', '1DUL', '1DUQ', '1E7K', '1E8O', '1EC6', '1EFO', '1EFW', '1EGK', '1EHZ', '1EIY', '1EQQ', '1ET4', '1EUQ', '1EUY', '1EVV', '1EXD', '1F1T', '1F27', '1F7U', '1F7V', '1F7Y', '1F8V', '1FEU', '1FFK', '1FFY', '1FFZ', '1FG0', '1FIR', '1FIX', '1FJG', '1FKA', '1FUF', '1FXL', '1G1X', '1G2E', '1G2J', '1G4Q', '1G59', '1GAX', '1GID', '1GSG', '1GTF', '1GTN', '1GTR', '1GTS', '1H2C', '1H2D', '1H38', '1H3E', '1H4Q', '1H4S', '1HC8', '1HMH', '1HNW', '1HNX', '1HNZ', '1HQ1', '1HR0', '1HR2', '1HYS', '1I5L', '1I6H', '1I6U', '1I7J', '1I94', '1I9V', '1I9X', '1IBK', '1IBL', '1IBM', '1IK5', '1IL2', '1IVS', '1J1U', '1J2B', '1J5A', '1J5E', '1J6S', '1J7T', '1J8G', '1J9H', '1JB8', '1JBR', '1JBS', '1JBT', '1JID', '1JJ2', '1JZV', '1JZX', '1JZY', '1K01', '1K73', '1K8A', '1K8W', '1K9M',

In [16]:
for filename in str_files:
    full_path= os.path.join(str_dir, filename+'.cif')
    if os.path.isfile(full_path):
        print ('FILE ALREADY EXISTS! NOT DOWNLOADING!')
    else: 
        print ('Downloading '+ filename)
        url = "http://www.rcsb.org/pdb/files/%s.cif" %filename
        call (["curl","-L","-O","-s",url])

FILE ALREADY EXISTS! NOT DOWNLOADING!
FILE ALREADY EXISTS! NOT DOWNLOADING!
FILE ALREADY EXISTS! NOT DOWNLOADING!
FILE ALREADY EXISTS! NOT DOWNLOADING!
FILE ALREADY EXISTS! NOT DOWNLOADING!
FILE ALREADY EXISTS! NOT DOWNLOADING!
FILE ALREADY EXISTS! NOT DOWNLOADING!
FILE ALREADY EXISTS! NOT DOWNLOADING!
FILE ALREADY EXISTS! NOT DOWNLOADING!
FILE ALREADY EXISTS! NOT DOWNLOADING!
FILE ALREADY EXISTS! NOT DOWNLOADING!
FILE ALREADY EXISTS! NOT DOWNLOADING!
FILE ALREADY EXISTS! NOT DOWNLOADING!
FILE ALREADY EXISTS! NOT DOWNLOADING!
FILE ALREADY EXISTS! NOT DOWNLOADING!
FILE ALREADY EXISTS! NOT DOWNLOADING!
FILE ALREADY EXISTS! NOT DOWNLOADING!
FILE ALREADY EXISTS! NOT DOWNLOADING!
FILE ALREADY EXISTS! NOT DOWNLOADING!
FILE ALREADY EXISTS! NOT DOWNLOADING!
FILE ALREADY EXISTS! NOT DOWNLOADING!
FILE ALREADY EXISTS! NOT DOWNLOADING!
FILE ALREADY EXISTS! NOT DOWNLOADING!
FILE ALREADY EXISTS! NOT DOWNLOADING!
FILE ALREADY EXISTS! NOT DOWNLOADING!
FILE ALREADY EXISTS! NOT DOWNLOADING!
FILE ALREADY

# copy characterized files

In [48]:
def copy_existing_files(file_list, src_dir, dest_dir):
    """
    Check each file in file_list. If the file exists in src_dir,
    copy it to dest_dir.
    """
    # Ensure the destination directory exists; if not, create it.
    if not os.path.exists(dest_dir):
        os.makedirs(dest_dir)

    # Iterate over the list of files
    for file_name in file_list:
        src_file = os.path.join(src_dir, file_name)
        if os.path.isfile(src_file):
            # Copy the file to the destination directory
            shutil.copy(src_file, dest_dir)
            print(f"Copied: {file_name}")
        else:
            print(f"File not found: {file_name}")

In [49]:
# Example usage:
str_list = [PID + '.cif' for PID in D6['PDB_ID']]
chr_list= [PID + '-dssr.json' for PID in D6['PDB_ID']]
source_directory = '/Users/sharear/Documents/sky/RESEARCH_&_GRAD_SCHOOL/Penn_State/anionic_G_U/get_all_RNA/new_data_April_2023/2nd_version_algorithm_removing_redundancy/anionic_GU_pipeline/anionic_GU_pipeline_updated/identifying_shifted_wobbles/all_cifs'         # Change to your source directory
destination_directory = '/Users/sharear/Documents/sky/RESEARCH_&_GRAD_SCHOOL/Penn_State/cifs_jsons/str_files' # Change to your destination directory

copy_existing_files(str_list, source_directory, destination_directory)
copy_existing_files(chr_list, source_directory, destination_directory)

Copied: 157D.cif
Copied: 165D.cif
Copied: 1A34.cif
Copied: 1A9N.cif
Copied: 1AQ3.cif
Copied: 1AQ4.cif
Copied: 1ASY.cif
Copied: 1ASZ.cif
Copied: 1AV6.cif
Copied: 1B23.cif
Copied: 1B2M.cif
Copied: 1B7F.cif
Copied: 1BMV.cif
Copied: 1BR3.cif
Copied: 1C0A.cif
Copied: 1C9S.cif
Copied: 1CGM.cif
Copied: 1CSL.cif
Copied: 1CVJ.cif
Copied: 1CWP.cif
Copied: 1D4R.cif
Copied: 1DDL.cif
Copied: 1DDY.cif
Copied: 1DFU.cif
Copied: 1DI2.cif
Copied: 1DK1.cif
Copied: 1DQF.cif
Copied: 1DQH.cif
Copied: 1DRZ.cif
Copied: 1DUH.cif
Copied: 1DUL.cif
Copied: 1DUQ.cif
Copied: 1E7K.cif
Copied: 1E8O.cif
Copied: 1EC6.cif
Copied: 1EFO.cif
Copied: 1EFW.cif
Copied: 1EGK.cif
Copied: 1EHZ.cif
Copied: 1EIY.cif
Copied: 1EQQ.cif
Copied: 1ET4.cif
Copied: 1EUQ.cif
Copied: 1EUY.cif
Copied: 1EVV.cif
Copied: 1EXD.cif
Copied: 1F1T.cif
Copied: 1F27.cif
Copied: 1F7U.cif
Copied: 1F7V.cif
Copied: 1F7Y.cif
Copied: 1F8V.cif
Copied: 1FEU.cif
Copied: 1FFK.cif
Copied: 1FFY.cif
Copied: 1FFZ.cif
Copied: 1FG0.cif
Copied: 1FIR.cif
Copied: 1FIX.c

Copied: 2IL9.cif
Copied: 2IX1.cif
Copied: 2IY5.cif
Copied: 2IZ8.cif
Copied: 2IZ9.cif
Copied: 2IZM.cif
Copied: 2IZN.cif
Copied: 2J0Q.cif
Copied: 2J0S.cif
Copied: 2JEA.cif
Copied: 2JJA.cif
Copied: 2JLT.cif
Copied: 2JLU.cif
Copied: 2JLV.cif
Copied: 2JLW.cif
Copied: 2JLX.cif
Copied: 2JLY.cif
Copied: 2JLZ.cif
Copied: 2NOK.cif
Copied: 2NPY.cif
Copied: 2NPZ.cif
Copied: 2NQP.cif
Copied: 2NUE.cif
Copied: 2NUF.cif
Copied: 2NUG.cif
Copied: 2NVQ.cif
Copied: 2NVT.cif
Copied: 2NZ4.cif
Copied: 2O3V.cif
Copied: 2O3W.cif
Copied: 2O3X.cif
Copied: 2O3Y.cif
Copied: 2O44.cif
Copied: 2O5I.cif
Copied: 2O5J.cif
Copied: 2OE5.cif
Copied: 2OE6.cif
Copied: 2OE8.cif
Copied: 2OEU.cif
Copied: 2OGM.cif
Copied: 2OIH.cif
Copied: 2OIJ.cif
Copied: 2OIU.cif
Copied: 2OIY.cif
Copied: 2OJ0.cif
Copied: 2OJ3.cif
Copied: 2OTJ.cif
Copied: 2OTL.cif
Copied: 2OUE.cif
Copied: 2OZB.cif
Copied: 2P7D.cif
Copied: 2P7E.cif
Copied: 2P7F.cif
Copied: 2PJP.cif
Copied: 2PLY.cif
Copied: 2PN3.cif
Copied: 2PN4.cif
Copied: 2PO1.cif
Copied: 2PPB.c

Copied: 3PF5.cif
Copied: 3PIO.cif
Copied: 3PIP.cif
Copied: 3PKM.cif
Copied: 3PLA.cif
Copied: 3PO2.cif
Copied: 3PO3.cif
Copied: 3PTX.cif
Copied: 3PU0.cif
Copied: 3PU1.cif
Copied: 3PU4.cif
Copied: 3Q0L.cif
Copied: 3Q0M.cif
Copied: 3Q0N.cif
Copied: 3Q0O.cif
Copied: 3Q0P.cif
Copied: 3Q0Q.cif
Copied: 3Q0R.cif
Copied: 3Q0S.cif
Copied: 3Q2T.cif
Copied: 3Q3Z.cif
Copied: 3Q50.cif
Copied: 3Q51.cif
Copied: 3QG9.cif
Copied: 3QGB.cif
Copied: 3QGC.cif
Copied: 3QJJ.cif
Copied: 3QJL.cif
Copied: 3QJP.cif
Copied: 3QRP.cif
Copied: 3QRR.cif
Copied: 3QSU.cif
Copied: 3QSY.cif
Copied: 3R1C.cif
Copied: 3R1D.cif
Copied: 3R1E.cif
Copied: 3R1H.cif
Copied: 3R1L.cif
Copied: 3R2C.cif
Copied: 3R2D.cif
Copied: 3R4F.cif
Copied: 3R9W.cif
Copied: 3R9X.cif
Copied: 3RC8.cif
Copied: 3RER.cif
Copied: 3RG5.cif
Copied: 3RKF.cif
Copied: 3RTJ.cif
Copied: 3RW6.cif
Copied: 3RZD.cif
Copied: 3RZO.cif
Copied: 3S14.cif
Copied: 3S15.cif
Copied: 3S16.cif
Copied: 3S17.cif
Copied: 3S1M.cif
Copied: 3S1N.cif
Copied: 3S1Q.cif
Copied: 3S1R.c

Copied: 4QYZ.cif
Copied: 4R3I.cif
Copied: 4R4P.cif
Copied: 4R4V.cif
Copied: 4R8I.cif
Copied: 4RBQ.cif
Copied: 4RBY.cif
Copied: 4RBZ.cif
Copied: 4RC0.cif
Copied: 4RCJ.cif
Copied: 4RCM.cif
Copied: 4RDX.cif
Copied: 4RGE.cif
Copied: 4RGF.cif
Copied: 4RJ1.cif
Copied: 4RKV.cif
Copied: 4RMO.cif
Copied: 4RNE.cif
Copied: 4RUM.cif
Copied: 4RWN.cif
Copied: 4RWO.cif
Copied: 4RWP.cif
Copied: 4RZD.cif
Copied: 4S2X.cif
Copied: 4S2Y.cif
Copied: 4S3N.cif
Copied: 4TNA.cif
Copied: 4TRA.cif
Copied: 4TS0.cif
Copied: 4TS2.cif
Copied: 4TU0.cif
Copied: 4TUE.cif
Copied: 4TUW.cif
Copied: 4TUX.cif
Copied: 4TV0.cif
Copied: 4TVX.cif
Copied: 4TYW.cif
Copied: 4TYY.cif
Copied: 4TZ0.cif
Copied: 4TZ6.cif
Copied: 4TZX.cif
Copied: 4TZY.cif
Copied: 4U1U.cif
Copied: 4U1V.cif
Copied: 4U20.cif
Copied: 4U24.cif
Copied: 4U25.cif
Copied: 4U26.cif
Copied: 4U27.cif
Copied: 4U34.cif
Copied: 4U35.cif
Copied: 4U37.cif
Copied: 4U38.cif
Copied: 4U3L.cif
Copied: 4U3M.cif
Copied: 4U3N.cif
Copied: 4U3O.cif
Copied: 4U3P.cif
Copied: 4U3R.c

Copied: 5HCQ.cif
Copied: 5HCR.cif
Copied: 5HD1.cif
Copied: 5HJZ.cif
Copied: 5HK0.cif
Copied: 5HKC.cif
Copied: 5HN2.cif
Copied: 5HNJ.cif
Copied: 5HNQ.cif
Copied: 5HO4.cif
Copied: 5HP2.cif
Copied: 5HP3.cif
Copied: 5HR6.cif
Copied: 5HR7.cif
Copied: 5HSW.cif
Copied: 5I4A.cif
Copied: 5I4L.cif
Copied: 5I9D.cif
Copied: 5I9F.cif
Copied: 5I9G.cif
Copied: 5I9H.cif
Copied: 5IB7.cif
Copied: 5IB8.cif
Copied: 5IBB.cif
Copied: 5ID6.cif
Copied: 5IP2.cif
Copied: 5IQR.cif
Copied: 5IT8.cif
Copied: 5IWA.cif
Copied: 5J01.cif
Copied: 5J02.cif
Copied: 5J30.cif
Copied: 5J3C.cif
Copied: 5J4B.cif
Copied: 5J4C.cif
Copied: 5J4D.cif
Copied: 5J5B.cif
Copied: 5J7L.cif
Copied: 5J88.cif
Copied: 5J8A.cif
Copied: 5J8B.cif
Copied: 5J91.cif
Copied: 5JAJ.cif
Copied: 5JB2.cif
Copied: 5JBG.cif
Copied: 5JC3.cif
Copied: 5JC7.cif
Copied: 5JC9.cif
Copied: 5JCF.cif
Copied: 5JCH.cif
Copied: 5JEA.cif
Copied: 5JJI.cif
Copied: 5JJK.cif
Copied: 5JJL.cif
Copied: 5JJU.cif
Copied: 5JRC.cif
Copied: 5JS1.cif
Copied: 5JS2.cif
Copied: 5JUP.c

...

Copied: 6UPX-dssr.json
Copied: 6UPY-dssr.json
Copied: 6UPZ-dssr.json
Copied: 6UQ2-dssr.json
Copied: 6UQ3-dssr.json
Copied: 6UTV-dssr.json
Copied: 6UV1-dssr.json
Copied: 6UV2-dssr.json
Copied: 6UV3-dssr.json
Copied: 6UV4-dssr.json
Copied: 6V39-dssr.json
Copied: 6V3A-dssr.json
Copied: 6V3B-dssr.json
Copied: 6V3D-dssr.json
Copied: 6V4X-dssr.json
Copied: 6V9B-dssr.json
Copied: 6V9D-dssr.json
Copied: 6V9Q-dssr.json
Copied: 6VBW-dssr.json
Copied: 6VEM-dssr.json
Copied: 6VFF-dssr.json
Copied: 6VLZ-dssr.json
Copied: 6VM6-dssr.json
Copied: 6VMI-dssr.json
Copied: 6VMY-dssr.json
Copied: 6VPC-dssr.json
Copied: 6VQV-dssr.json
Copied: 6VQW-dssr.json
Copied: 6VQX-dssr.json
Copied: 6VRB-dssr.json
Copied: 6VRC-dssr.json
Copied: 6VRD-dssr.json
Copied: 6VUH-dssr.json
Copied: 6VUI-dssr.json
Copied: 6VWL-dssr.json
Copied: 6VWM-dssr.json
Copied: 6VWN-dssr.json
Copied: 6VWT-dssr.json
Copied: 6VWV-dssr.json
Copied: 6W11-dssr.json
Copied: 6W2S-dssr.json
Copied: 6W2T-dssr.json
Copied: 6W5C-dssr.json
Copied: 6W6

Copied: 7ELH-dssr.json
Copied: 7ELM-dssr.json
Copied: 7ELN-dssr.json
Copied: 7ELP-dssr.json
Copied: 7ELQ-dssr.json
Copied: 7ELR-dssr.json
Copied: 7ELS-dssr.json
Copied: 7ENI-dssr.json
Copied: 7EOG-dssr.json
Copied: 7EOH-dssr.json
Copied: 7EOI-dssr.json
Copied: 7EOJ-dssr.json
Copied: 7EOK-dssr.json
Copied: 7EOL-dssr.json
Copied: 7EOM-dssr.json
Copied: 7EON-dssr.json
Copied: 7EOO-dssr.json
Copied: 7EOP-dssr.json
Copied: 7EQG-dssr.json
Copied: 7EQJ-dssr.json
Copied: 7EU0-dssr.json
Copied: 7EU9-dssr.json
Copied: 7EVO-dssr.json
Copied: 7EWQ-dssr.json
File not found: 7EXA-dssr.json
Copied: 7EZ0-dssr.json
Copied: 7EZ2-dssr.json
Copied: 7F0D-dssr.json
Copied: 7F1M-dssr.json
Copied: 7F36-dssr.json
Copied: 7F3I-dssr.json
Copied: 7F3J-dssr.json
Copied: 7F3K-dssr.json
Copied: 7F3L-dssr.json
Copied: 7F5S-dssr.json
Copied: 7F8Z-dssr.json
File not found: 7FGI-dssr.json
Copied: 7JIL-dssr.json
Copied: 7JJD-dssr.json
Copied: 7JJE-dssr.json
Copied: 7JJF-dssr.json
Copied: 7JJU-dssr.json
Copied: 7JNH-dssr.

Copied: 7QH7-dssr.json
Copied: 7QI4-dssr.json
File not found: 7QI5-dssr.json
File not found: 7QI6-dssr.json
Copied: 7QIW-dssr.json
Copied: 7QIX-dssr.json
Copied: 7QIY-dssr.json
Copied: 7QIZ-dssr.json
Copied: 7QP2-dssr.json
Copied: 7QQ3-dssr.json
Copied: 7QQO-dssr.json
Copied: 7QQP-dssr.json
Copied: 7QQQ-dssr.json
Copied: 7QQR-dssr.json
Copied: 7QQS-dssr.json
Copied: 7QQT-dssr.json
Copied: 7QQU-dssr.json
Copied: 7QQV-dssr.json
Copied: 7QQW-dssr.json
Copied: 7QQX-dssr.json
Copied: 7QQZ-dssr.json
Copied: 7QR0-dssr.json
Copied: 7QR1-dssr.json
Copied: 7QR3-dssr.json
Copied: 7QR4-dssr.json
Copied: 7QR5-dssr.json
Copied: 7QR7-dssr.json
Copied: 7QR8-dssr.json
Copied: 7QSH-dssr.json
Copied: 7QTL-dssr.json
Copied: 7QTN-dssr.json
File not found: 7QTT-dssr.json
Copied: 7QUA-dssr.json
Copied: 7QV1-dssr.json
Copied: 7QV2-dssr.json
Copied: 7QVP-dssr.json
Copied: 7QWQ-dssr.json
Copied: 7QWR-dssr.json
Copied: 7QWS-dssr.json
Copied: 7QXA-dssr.json
Copied: 7R0E-dssr.json
Copied: 7R1F-dssr.json
File not f

Copied: 7ZPQ-dssr.json
Copied: 7ZQ5-dssr.json
Copied: 7ZQ6-dssr.json
File not found: 7ZRZ-dssr.json
Copied: 7ZS5-dssr.json
Copied: 7ZTA-dssr.json
Copied: 7ZUX-dssr.json
Copied: 7ZW0-dssr.json
Copied: 8A22-dssr.json
Copied: 8A3D-dssr.json
File not found: 8A3L-dssr.json
File not found: 8A3W-dssr.json
File not found: 8A3Y-dssr.json
File not found: 8A40-dssr.json
Copied: 8A57-dssr.json
Copied: 8A5I-dssr.json
Copied: 8A63-dssr.json
File not found: 8A98-dssr.json
Copied: 8AAF-dssr.json
Copied: 8ABZ-dssr.json
File not found: 8ACB-dssr.json
File not found: 8ACC-dssr.json
Copied: 8AF0-dssr.json
Copied: 8AGT-dssr.json
Copied: 8AGU-dssr.json
Copied: 8AGV-dssr.json
Copied: 8AGW-dssr.json
Copied: 8AGX-dssr.json
Copied: 8AGZ-dssr.json
Copied: 8AKN-dssr.json
Copied: 8AM9-dssr.json
Copied: 8AMG-dssr.json
Copied: 8AMI-dssr.json
Copied: 8AMJ-dssr.json
Copied: 8AMK-dssr.json
Copied: 8AML-dssr.json
Copied: 8AMM-dssr.json
Copied: 8AMN-dssr.json
Copied: 8ANA-dssr.json
Copied: 8ANE-dssr.json
File not found: 

Copied: 8G60-dssr.json
Copied: 8G61-dssr.json
Copied: 8G6J-dssr.json
Copied: 8G6R-dssr.json
Copied: 8G6W-dssr.json
Copied: 8G6X-dssr.json
Copied: 8G6Y-dssr.json
File not found: 8G7P-dssr.json
File not found: 8G7Q-dssr.json
File not found: 8G7R-dssr.json
File not found: 8G7S-dssr.json
File not found: 8G7T-dssr.json
File not found: 8G90-dssr.json
File not found: 8G91-dssr.json
Copied: 8G9L-dssr.json
File not found: 8G9S-dssr.json
File not found: 8G9U-dssr.json
Copied: 8G9Z-dssr.json
File not found: 8GAM-dssr.json
File not found: 8GAN-dssr.json
Copied: 8GH6-dssr.json
Copied: 8GHU-dssr.json
File not found: 8GKH-dssr.json
Copied: 8GLP-dssr.json
Copied: 8GNA-dssr.json
Copied: 8GS2-dssr.json
Copied: 8GU6-dssr.json
File not found: 8GW1-dssr.json
Copied: 8GWB-dssr.json
Copied: 8GWE-dssr.json
Copied: 8GWF-dssr.json
Copied: 8GWG-dssr.json
Copied: 8GWI-dssr.json
File not found: 8GWK-dssr.json
File not found: 8GWM-dssr.json
Copied: 8GWN-dssr.json
Copied: 8GXB-dssr.json
Copied: 8GXC-dssr.json
Copied